In [5]:
%matplotlib inline
import libs.config
import libs.files as fh
import libs.pipeline as pipe
import libs.parse as p
import libs.twokenize as ark
import libs.resources as r
import numpy as np
import pandas as pd
import unidecode

import codecs

For the SemEval 2015, the task 10 part b is related to the Tweet sentiment analysis. The train and test dataset are the same used in 2013 and 2014

In [39]:
# read the train file
# tw_downloaded = "../1-Input/2014/SemEval2014-task9-test-B-gold.txt.error"
# original_file = "../1-Input/2014/SemEval2014-task9-test-B-gold-NEED-TWEET-DOWNLOAD.txt"
final_file = "../1-Input/2015/SemEval2015_test.csv"
gold_file = "../1-Input/2015/SemEval2015-task10-test-B-gold.txt"
sarcasm_file = "../1-Input/2015/SemEval2014-task9-test-B-sarcasm.txt"

# the task in SEMEVAL
file_type = 'B'

In [7]:
# read the file
test_feat, gold, test_tweets = pipe.create_features(final_file, file_type, '')
len(test_tweets)

Read 2389 rows from ../1-Input/2015/SemEval2015_test.csv


1394

In [8]:
# read the information
OUTPUT_DIR = '../3-Output/'
MODEL_OUT_DIR = '../6-Models'
CREATE_TOKENS_FILES = True
PROCESS_DIR = '../2-Processed'
rnd = 9000

# read the config file
cfg = fh.read_config_file("all.yaml")

# read the train file
file_name = "../1-Input/trainingData-B.tsv"
file_type = 'B'
train, labels, train_tweets = pipe.create_features(file_name, file_type, cfg)

# read the dev file
file_name = "../1-Input/devData-B.tsv"
file_type = 'B'
dev, dev_labels, dev_tweets = pipe.create_features(file_name, file_type, cfg)

Read 9683 rows from ../1-Input/trainingData-B.tsv
Read 1653 rows from ../1-Input/devData-B.tsv


In [9]:
# random seed for all the operations
rnd_seed = 9000

# join datasets
train_final = train_tweets[:]
train_final.extend(dev_tweets)

# convert the list into and array that can be indexed
train_labels = np.concatenate((np.array(labels),np.array(dev_labels)))
gold = np.array(gold)

# create the cleaned the tweets
train_clean, test_clean, vect = pipe.create_count_vec(train_final, test_tweets, tokenizer=pipe.tokenize_clean_raw, stop_words=pipe.stop_words)
train_data, test_data, _ = pipe.auto_select_features(pipe.chi2, 5, train_clean, train_labels, test_clean, gold)

train shape: (9576, 19847)
dev shape: (1394, 19847)
Final shape (9576, 993) (1394, 993)


In [10]:
# using negated tokens
train_neg_tokens = [pipe.tokenize_negate_clean_raw(t) for t in train_final]
test_neg_tokens = [pipe.tokenize_negate_clean_raw(t) for t in test_tweets]

# not using negated tokens
train_tokens = [pipe.tokenize_clean_raw(t) for t in train_final]
test_tokens = [pipe.tokenize_clean_raw(t) for t in test_tweets]

In [11]:
def decode_gene(ind):
    """
    Decode a gene to a information that can be processed.
    """
    return [r.lexs[idx] for idx, present in enumerate(ind) if present]

def select_features(lex, train_feat, labels, test_feat):
    """
    Select the most important features according to a given criteria.
    """
    tmp_train_feat, tmp_test_feat, _, _ = pipe.create_lex_vec(train_feat, test_feat)
    tmp_train_feat, tmp_test_feat, selector = pipe.auto_select_features(pipe.mutual_info_classif, lex.selection_percent, 
                                                                       tmp_train_feat, labels, 
                                                                       tmp_test_feat, None)
    return tmp_train_feat, tmp_test_feat

def create_ind_features(ind, base_train, base_test, train_tokens, test_tokens):
    """
    Given an individual, create the features according to its genes
    Params:
        ind: individual generated in the genetic algorithm
    Returns:
        train and test data as a scipy sparse matrix
    """
     # remove the last locus as it refers to the LIWC
    LIWC_gene = ind[-1]
    tmp_ind = ind[:-1]
    
    # load the lexicons according to the gens
    ind_lexs = decode_gene(tmp_ind)
    
    # for each of the lexicons, merge them
    final_train = base_train.copy()
    final_test = base_test.copy()

    for lex in ind_lexs:
#         try:
        print('Joining the lex {}'.format(lex.prefix))
        # create the features from the datasets tokens
        if False:
            train_feat = lex.process_lex(train_neg_tokens, use_best_features=True)
            test_feat = lex.process_lex(test_neg_tokens, use_best_features=True)
        else:
            train_feat = lex.process_lex(train_tokens, use_best_features=True)
            test_feat = lex.process_lex(test_tokens, use_best_features=True)

#         print len(train_feat), len(test_feat)
#             print  test_feat[:3]

        # select the top percent features if this is the case. Set the flag to create vector at save if selection occurs
        if lex.selection_percent:
            train_feat, test_feat = select_features(lex, train_feat, train_labels, test_feat)
        else:
            train_feat, test_feat, _,_ = pipe.create_lex_vec(train_feat, test_feat)

#         print train_data.shape, test_data.shape
        final_train, final_test = pipe.join_lex_features(final_train, train_feat, 
                                                         final_test, test_feat, 
                                                         verbose=True, create_vec=False)
#         except:
#             u.print_exception()
#             print 'error loading ind', ind
#             print lex

        # check if should add LIWC
    if LIWC_gene:
        final_train, final_test = pipe.join_lex_features(final_train, full_train_liwc, 
                                                        final_test, test_liwc, 
                                                        verbose=False, create_vec=True)   
        print final_train.shape, final_test.shape
    return final_train, final_test

In [12]:
# read best gene
df = pd.read_csv('../3-Output/population.csv', header=None, sep=';')
df.columns = ['Gene', 'Stats']

# create the best gene combination
best_individual = eval(df.iloc[0]['Gene'])
X_train, X_test = create_ind_features(best_individual, 
                                      train_data, test_data, 
                                      train_neg_tokens, test_neg_tokens)

Joining the lex BING
train data, lex and final shape:  (9576, 993) (9576, 7) (9576, 1000)
test data, lex and final shape:  (1394, 993) (1394, 7) (1394, 1000)
Joining the lex SWN
train data, lex and final shape:  (9576, 1000) (9576, 8) (9576, 1008)
test data, lex and final shape:  (1394, 1000) (1394, 8) (1394, 1008)
Joining the lex MSOL
Final shape (9576, 413) (1394, 413)
train data, lex and final shape:  (9576, 1008) (9576, 413) (9576, 1421)
test data, lex and final shape:  (1394, 1008) (1394, 413) (1394, 1421)
Joining the lex NRCHASH
train data, lex and final shape:  (9576, 1421) (9576, 7) (9576, 1428)
test data, lex and final shape:  (1394, 1421) (1394, 7) (1394, 1428)
Joining the lex TSLEX
train data, lex and final shape:  (9576, 1428) (9576, 7) (9576, 1435)
test data, lex and final shape:  (1394, 1428) (1394, 7) (1394, 1435)
Joining the lex WNA
Final shape (9576, 111) (1394, 111)
train data, lex and final shape:  (9576, 1435) (9576, 111) (9576, 1546)
test data, lex and final shape:

In [13]:
final_val_scores = pipe.run_multiple_class(X_train, train_labels, X_test, gold, 
                                           rnd_seed=rnd_seed, use_best_params=True,
                                           test_name='test')

In [25]:
pipe.pprint_results(final_val_scores, test_name='test')

In [27]:
final_scores = pipe.predict_test_multi_proc('LogisticRegression', X_train, train_labels, X_test, gold, test_name='test', rnd_seed=rnd)

In [33]:
final_scores

,train score,test score,f1_test_neg,f1_test_neu,f1_test_pos,train std,test std,f1_test_neg std,f1_test_neu std,f1_test_pos std
LogisticRegression,0.75076,0.59669,0.535433,0.690513,0.657948,0.000136,0.0,0.0,1.170278e-16,0.0


In [22]:
# create the output
cls = pipe.LogisticRegression(**pipe.lr_params)
cls.set_params(random_state=rnd_seed)
cls.fit(X_train, train_labels)
pred_test = cls.predict(X_test)

In [23]:
decode = {
    -1: "negative",
    0:"neutral",
    1: "positive"
}
pred_test_labels = [ decode[val] for val in pred_test]

### Generate final file for submission
The final submission has a specific format that needs to be followed so that it can be properly evaluated. As some tweets were not available, the prediction is made as neutral

In [35]:
# create dictionary with all the key/answers pair as we have missing text in the output
out_text = []
for tf,pred in zip(test_feat, pred_test_labels):
    out_text.append(u"NA\t{}\t{}\n".format(tf.uid, pred))

In [38]:
out_file = '../3-Output/prediction_2015.csv'
with codecs.open(out_file, 'w', 'utf-8') as f:
    f.writelines(out_text)

The final submission scored by SemEval 2014 script was :

| LiveJournal2014	| 	SMS2013	| 	Twitter2013	| 	Twitter2014	| 	Twitter2014Sarcasm	| 
|-------------------------------|--------|-------|-------|-------|
|71.11  | 65.89     | 58.51     | 56.21     |  42.05     |



Not a very good score.

### Sarcasm
2015 had a small sample of tweets that were ideintified as sarcasm. Checking how was the performance on this.

In [80]:
df = pd.read_csv(sarcasm_file, header=None, sep='\t')
sarcasm_tweets = df[0].apply(lambda x: unicode(x)).values

In [83]:
decode = {
    "negative":-1,
    "neutral":0,
    "positive":1,
    None:None
}
pred_sarcasm = []
gold_sarcasm = []
for tf,pred in zip(test_feat, pred_test_labels):
    if tf.sid in sarcasm_tweets:
        pred_sarcasm.append(decode[pred])
        gold_sarcasm.append(decode[tf.sent])
        
print len(pred_sarcasm)

36


In [82]:
'{0:0.2f}'.format(100.0*pipe.score_func(gold_sarcasm, pred_sarcasm))

'52.22'